In [1]:
import snorkel

In [2]:
import pandas as pd

In [3]:
crowd_df_1 =pd.read_csv('crowd/match_crowd_data.csv')
crowd_df_1.sample()

,Unnamed: 0,DB_ID,DO_ID,disease,Label_ID,relation,Section,Set_ID,Drug_Brand_Name,Active_ingredient,UNII_ID,Context,WordCount,Drug_name
251,100,DB00850,DOID_5419,schizophrenia,84141783-43f3-ddfd-e053-2991aa0a907a.xml,treatment,Indications,a827ab2d-ab1c-46eb-bc52-8bae5115a481,Perphenazine,PERPHENAZINE,FTA7XXY4EZ,Perphenazine tablets are indicated for use in ...,40,Perphenazine


In [4]:

crowd_df_1 = crowd_df_1[['DB_ID', 'DO_ID','disease', 'Label_ID','Set_ID', 'relation', 'Drug_name', 'Context','Section' ]]
crowd_df_1.sample()

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
582,DB00152,DOID_13725,beriberi,e8bd17b1-1729-4c98-9467-d8a74e1e5b6e.xml,5982cf83-01d6-4acb-8373-339079b8a76e,treatment,Thiamine,Thiamine Hydrochloride Injection is effective ...,Indications


In [5]:
crowd_df_plus = pd.read_csv('crowd/crowd_2nd.csv')
crowd_df_plus = crowd_df_plus[['DB_ID', 'DO_ID','disease', 'Label_ID', 'Set_ID','relation', 'Drug_name', 'Context','Section' ]]
crowd_df_plus.sample()

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
201,DB01234,DOID_2773,contact dermatitis,537cd838-05f2-44f4-aeb2-89c42f153e3c.xml,e5c28db5-efce-49a7-9e5f-12338d1f216b,treatment,DEXAMETHASONE,Control of severe or incapacitating allergic c...,Indications


In [6]:

crowd_df = crowd_df_1.append(crowd_df_plus)
len(crowd_df)

2185

In [7]:
crowd_df= crowd_df.drop_duplicates(subset=['Label_ID','DO_ID','Section'], keep='last')
len(crowd_df)

2162

In [8]:
unlabeled_df  = pd.read_csv('unlabeled/raw_cleaned.csv')
unlabeled_df = unlabeled_df[['DB_ID', 'DO_ID','disease', 'Label_ID', 'DrugName', 'Context','Section' ]]
len(unlabeled_df)

54040

In [9]:
unlabeled_df= unlabeled_df.rename(columns={'DrugName':'Drug_name'})

In [10]:
unlabeled_df.drop_duplicates(subset=['Context','DO_ID'], keep='first', inplace=True)
len(unlabeled_df)

39810

In [11]:
experts = pd.read_csv('expert/clean_expert_resolved.csv')
experts = experts[['DB_ID', 'DO_ID','disease', 'Label_ID', 'Set_ID','relation', 'Drug_name', 'Context','Section']]
experts.sample()

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
31,DB00999,DOID_10763,HYPERTENSION,414a7bf2-7435-0d18-e054-00144ff8d46c.xml,414a7bf2-7434-0d18-e054-00144ff8d46c,Indication: Treatment,Hydrochlorothiazide,Hydrochlorothiazide Tablets are indicated as a...,Indications


In [12]:
len(experts)

284

In [13]:
def rep(text):
    a = text.replace('Contraindication', 'contraindication')
    b = a.replace('Indication: Treatment', 'treatment')
    c = b.replace('Indication: Symptomatic Relief', 'relief')
    d = c.replace('Effect', 'effect')
    return d

def lower (x):
    return x.lower()

# remove punctuation
def remove_symbols(x):
    for char in x:
            if char not in ['1','2','3','4','5','6','7','8','9','q','Q','w','W', 'e','E','r','R','t','T','y','Y','U','u','i','I','O','o','P','p','A','a','S','s','D','d','F','f','G','g','H','h','J','j','K','k','L','l','Z','z','X','x','C','c','V','v','B','b','N','n','M','m']:
                x = x.replace(char, ' ')
    return x

def check(x, text):
    return x in text

def remove_xml_symbols(text):
    text1 = text.replace('\n', '  ')
    clean_text = text1.replace('\r', '  ')
    return clean_text

def remove_spaces(text):
    clean_text = ' '.join(text.split())
    return clean_text

experts['relation'] =experts['relation'].apply(rep)

experts['Context'] = experts['Context'].apply(remove_xml_symbols)
experts['Context'] = experts['Context'].apply(remove_symbols)
experts['Context'] = experts['Context'].apply(remove_spaces)
experts['Context'] = experts['Context'].apply(lower)

experts['disease'] = experts['disease'].apply(remove_xml_symbols)
experts['disease'] = experts['disease'].apply(remove_symbols)
experts['disease'] = experts['disease'].apply(remove_spaces)
experts['disease'] = experts['disease'].apply(lower)


In [14]:
crowd_df['Context'] = crowd_df['Context'].apply(remove_xml_symbols)
crowd_df['Context'] = crowd_df['Context'].apply(remove_symbols)
crowd_df['Context'] = crowd_df['Context'].apply(remove_spaces)
crowd_df['Context'] = crowd_df['Context'].apply(lower)

crowd_df['disease'] = crowd_df['disease'].apply(remove_xml_symbols)
crowd_df['disease'] = crowd_df['disease'].apply(remove_symbols)
crowd_df['disease'] = crowd_df['disease'].apply(remove_spaces)
crowd_df['disease'] = crowd_df['disease'].apply(lower)



In [15]:
crowd_df.sample(2)

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
1277,DB00693,DOID_4481,allergic rhinitis,362ba67f-abf9-4210-a994-3814ca7bbb06.xml,ebb3883c-71f6-4fd0-a7e6-0ba8e1136dd9,effect,Fluorescein,fluorescite r see warnings and precautions 5 1...,Contraindications
85,DB00877,DOID_3319,lymphangioleiomyomatosis,26c745ce-ed4e-4b5f-9704-b6f3842c7881.xml,5908cd1a-fc5a-462f-99ed-1d8983e253c9,treatment,SIROLIMUS,sirolimus is an mtor inhibitor immunosuppressa...,Indications


In [16]:
test_df = experts
test_df.sample(4)

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
137,DB00829,DOID_11338,tetanus,4e800d0d-2181-49b1-a2c8-4c6c49edd83a.xml,41044928-dd1f-40bf-1fa6-709dff559124,relief,Diazepam,diazepam is indicated for the management of an...,Indications
86,DB05271,DOID_14330,parkinson s disease,a6964835-dc96-4b22-8b64-464a42f6cdf2.xml,939e28c5-f3a9-42c0-9a2d-8d471d82a6e0,treatment,Rotigotine,neupro is a dopamine agonist indicated for the...,Indications
68,DB08882,DOID_9352,type 2 diabetes mellitus,1135e6c8-c2ae-d958-20ba-6c1eda889aa2.xml,c797ea5c-cab7-494b-9044-27eba0cfe40f,treatment,Linagliptin,tradjenta is a dipeptidyl peptidase 4 dpp 4 in...,Indications
228,DB00989,DOID_3042,allergic contact dermatitis,35787807-49b2-6034-dcb0-36037574fb42.xml,ca943349-53ea-2b6e-53c3-0265b1a81f8a,effect,Rivastigmine,rivastigmine tartrate capsules are contraindic...,Contraindications


In [17]:
experts = experts.drop_duplicates(subset=['Context','DO_ID'], keep='last')
len(experts)

282

In [18]:
on_col =['DO_ID', 'Label_ID', 'Set_ID','Section']


In [19]:
train_df = crowd_df[~crowd_df[on_col].apply(tuple,1).isin(experts[on_col].apply(tuple,1))]
len(train_df)

1881

In [20]:
test_df =crowd_df.merge(experts[on_col], on=on_col)
len(test_df)

281

In [21]:
test_df.head()

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
0,DB00657,DOID_10824,malignant hypertension,976f404f-4857-4bad-9a3c-9bf3d5ca9f5b.xml,c6822ce4-e741-4366-b72a-5f5146354f97,treatment,Mecamylamine,for the management of moderately severe to sev...,Indications
1,DB00401,DOID_10763,hypertension,ce733b39-7857-4538-92d6-1c68a4e4eb75.xml,a6a8a8ba-3966-4f37-a58b-44d3b05bdc4e,treatment,Nisoldipine,nisoldipine extended release tablets are indic...,Indications
2,DB00612,DOID_10763,hypertension,c818c38d-ee37-4ceb-b078-611483b4f743.xml,c818c38d-ee37-4ceb-b078-611483b4f743,treatment,Bisoprolol,bisoprolol fumarate tablets usp are indicated ...,Indications
3,DB00960,DOID_10763,hypertension,d4078b63-30ad-435b-a081-4d90562962b5.xml,b5a5a1e9-ee9b-41b1-b2d5-5f5834b816a7,treatment,Pindolol,pindolol tablets are indicated in the manageme...,Indications
4,DB00876,DOID_10763,hypertension,ab531d28-345b-4113-9fde-00cd63ad5b1a.xml,64fbc500-3b77-4c9c-8576-b3ec5f6655a5,treatment,Eprosartan,eprosartan mesylate tablets are indicated for ...,Indications


In [22]:
train_df =unlabeled_df.append(train_df)
print (len(train_df))
train_df.drop_duplicates(subset=['Context','DO_ID'], keep='last', inplace=True)
print (len(train_df))

41691
40516


/home/jovyan/.local/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [23]:
dev_df = test_df

In [40]:
import numpy as np
indx = dev_df.sample(140).index
dev_df.loc[indx,'relation'] = np.nan
#temp_df['relation'] = 

In [41]:
dev_df.sample(10)

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
20,DB00187,DOID_0060224,atrial fibrillation,e296498d-530b-4346-af09-43008d65c46a.xml,d8e97b4a-0846-4a3f-872f-1893eeb25175,relief,Esmolol,1 1 supraventricular tachycardia or noncompens...,Indications
73,DB00541,DOID_9952,acute lymphocytic leukemia,768cd59b-38a2-4af5-ab91-4d46ae808ab3.xml,a1869e23-b93b-4407-843b-4b830319d82e,treatment,Vincristine,marqibo is a vinca alkaloid indicated for the ...,Indications
187,DB09126,DOID_305,carcinoma,7bd326f0-5c4c-2c4a-e053-2a91aa0a2752.xml,ac6918ff-9dc3-40d9-8619-54b26e4bdfc8,treatment,Chorionic Gonadotropin (Human),precocious puberty prostatic carcinoma or othe...,Contraindications
88,DB05271,DOID_14330,parkinson s disease,a6964835-dc96-4b22-8b64-464a42f6cdf2.xml,939e28c5-f3a9-42c0-9a2d-8d471d82a6e0,treatment,Rotigotine,neupro is a dopamine agonist indicated for the...,Indications
194,DB01219,DOID_2237,hepatitis,cbece5e5-72b3-433f-84ae-03013ce93454.xml,ade5fe9c-7601-490a-ac6f-31e63fbd3f8d,relief,Dantrolene,active hepatic disease such as hepatitis and c...,Contraindications
165,DB01050,DOID_7148,rheumatoid arthritis,d2e35cfc-1c17-456f-8608-5d2c6f7bcb02.xml,758eeb45-e599-4c1a-969d-8fa33f27bf14,NaN,Ibuprofen,carefully consider the potential benefits and ...,Indications
11,DB00542,DOID_10763,hypertension,8fdc8bda-635a-401f-a9b5-8a2a514b429e.xml,8fdc8bda-635a-401f-a9b5-8a2a514b429e,NaN,Benazepril,benazepril hydrochloride tablets usp are indic...,Indications
188,DB14713,DOID_2921,glomerulonephritis,922740f6-e540-49cc-8ecd-a705913fd8c4.xml,8513207e-b55f-417b-9473-af785146a543,NaN,Inotersen,tegsedi is contraindicated in patients with pl...,Contraindications
235,DB00524,DOID_2983,anuria,20d39fb1-12c2-4010-8010-ef55d8f48a08.xml,78105e5c-5d36-426c-9f79-f2e085460b75,contraindication,Metolazone,anuria hepatic coma or precoma known allergy o...,Contraindications
168,DB01345,DOID_0060319,cardiac arrest,09893986-19e0-4364-93ba-5fbc7e7d3eed.xml,d2c923d5-85c1-4bbb-9f93-556080a5ef87,relief,Potassium cation,urocit r in patients with hyperkalemia or who ...,Contraindications


In [42]:
len(train_df)

40516

In [43]:
dev_df.merge(experts, on=['Label_ID','DO_ID'])[['relation_x','relation_y']]

,relation_x,relation_y
0,NaN,treatment
1,treatment,treatment
2,treatment,treatment
3,treatment,treatment
4,NaN,treatment
...,...,...
276,effect,contraindication
277,NaN,contraindication
278,effect,contraindication
279,NaN,contraindication


In [44]:
Y_dev = dev_df.merge(experts, on=['Label_ID','DO_ID'])['relation_y']

In [45]:
classes_y = {
'effect': 0,
'contraindication': 1,
'relief': 2,
'treatment':3 
}
Y_dev =Y_dev.map(classes_y)

In [46]:
from snorkel.labeling import LFAnalysis

In [47]:
my_phrases = { 'effect': 
["hypersensitivity reactions",
"associated with the risk of",
"to the risk of",
"a high risk for",
"a high risk of",
"high incidence of", 
"higher incidence of", 
" cause ",
" causes ",
"symptoms occure",
"teratogenic",
"site reaction",
"the risk of development",
"is associated with a risk of",
"symptoms of the poisoning",
"symptoms of poisoning" 
],

 'contraindication': 
["not administrated to",
"contraindicated in",
"contraindicatedin",
"should not be used",
"is contraindication for",
"is contraindication when",
"is contraindicated when",
"must not be used for",
"do not administer",
"should not initiate",
"not be administered to",
"do not initiate patients",
"contraindication for",
"should not be given",
"do not use"],
              
 'relief': 
["relief of the signs",
"relief of signs",
"relief of symptoms",
"relief of the symptoms",
"help",
"helps",
"relief of signs and symptoms of",
"reduction of symptoms of",
"treatment of the symptoms of",
"for the relief",
"management of the signs and symptoms of"], 

 'treatment':
[" indicated for the treatment of",
" indicated in the management of",
" indicated for the management of",
"for the management of",
"management of",
" indicated for the maintenance of remission", 
"or the treatment of",
"in the treatment of",
" indicated as",
" indicated in",
"be effective",
"active treatment of",
" indicated for",
"treatment of",
" indicated as an adjunct",
" indicated for use in the treatment of", 
" indicated for the intermittent treatment", 
" indicated to reduce the rate of",
" indicated for the rapid control",
" indicated for the control",
"reduce the risk of",
" indicated as adjunctive treatment",
"for the treatment of",
" indicated as an adjunct"]
           }

phrases = pd.DataFrame({ key:pd.Series(value) for key, value in my_phrases.items() })

effect = { 'phrases': 
["hypersensitivity reactions",
"associated with the risk of",
"to the risk of",
"a high risk for",
"a high risk of",
"high incidence of", 
"higher incidence of", 
" cause ",
" causes ",
"symptoms occure",
"teratogenic",
"site reaction",
"the risk of development",
"is associated with a risk of",
"symptoms of the poisoning",
"symptoms of poisoning" 
]}
effect_df = pd.DataFrame(effect)

 
contraindication = {'phrases': 
["not administrated to",
"contraindicated in",
"contraindicatedin",
"should not be used",
"is contraindication for",
"is contraindication when",
"is contraindicated when",
"must not be used for",
"do not administer",
"should not initiate",
"not be administered to",
"do not initiate patients",
"contraindication for",
"should not be given",
"do not use"]}
contraindication_df = pd.DataFrame(contraindication)
              
relief = {'phrases': 
["relief of the signs",
"relief of signs",
"relief of symptoms",
"relief of the symptoms",
"help",
"helps",
"relief of signs and symptoms of",
"reduction of symptoms of",
"treatment of the symptoms of",
"for the relief",
"management of the signs and symptoms of"]} 
relief_df = pd.DataFrame(relief)

treatment = {'phrases':
[" indicated for the treatment of",
" indicated in the management of",
" indicated for the management of",
"for the management of",
"management of",
" indicated for the maintenance of remission", 
"or the treatment of",
"in the treatment of",
" indicated as",
" indicated in",
"be effective",
"active treatment of",
" indicated for",
"treatment of",
" indicated as an adjunct",
" indicated for use in the treatment of", 
" indicated for the intermittent treatment", 
" indicated to reduce the rate of",
" indicated for the rapid control",
" indicated for the control",
"reduce the risk of",
" indicated as adjunctive treatment",
"for the treatment of"]}
treatment_df = pd.DataFrame(treatment)

In [48]:
from snorkel.labeling import labeling_function
from snorkel.preprocess import preprocessor


def check(x, string):
    return x in string


def find_phrases(text):
    # Find which phrases appear in the sentence
    class_effect = effect_df['phrases'].apply(lambda num : check(num,text))
    class_contraindication = contraindication_df['phrases'].apply(lambda num : check(num,text))
    class_relief = relief_df['phrases'].apply(lambda num : check(num,text))
    class_treatment = treatment_df['phrases'].apply(lambda num : check(num,text))
    
    #Create a dataset from them
    classes = {
    'effect': class_effect,
    'contraindication': class_contraindication,
    'relief': class_relief,
    'treatment':class_treatment 
    }

    classes = pd.DataFrame(classes)
    
    a = class_effect.index[class_effect == True].tolist()
    b = class_contraindication.index[class_contraindication == True].tolist()
    c = class_relief.index[class_relief == True].tolist()
    d = class_treatment.index[class_treatment == True].tolist()
    return classes, a, b, c, d
                                                 

# return the position of a phrase/word in a sentence
def pos_of_phrase(string, substring):
    substring_length = len(substring)    
    def recurse(locations_found, start):
        location = string.find(substring, start)
        if location != -1:
            return recurse(locations_found + [location], location+substring_length)
        else:
            return locations_found

    return recurse([], 0)

# return the minimum distance of all phrase/word from the target disease
def get_diff_of_pos(classes, column, pos, string):
    name = classes.columns[column]
    my_col = classes[name]
    x = [k for k,v in enumerate(my_col) if v == True]
    
    candidate = []
    distances = []
    for i in range(len(x)):
        word = phrases[classes.columns[column]][x[i]]
        
        for i in range(len(pos_of_phrase(string, word))):
            distances.append( abs(pos -   pos_of_phrase(string, word)[i]) )

    return min(distances)


@labeling_function()
def phrases_claas(x):
    
    text = x.Context

    classes, a, b, c, d =find_phrases(text)


    # Check if you have unique class and label based on that 
    if len(a)!=0 and len(b)==len(c)==len(d)==0:
        #print('effect')
        return RELATION_CLASSES['effect']
    elif len(b)!=0 and len(a)==len(c)==len(d)==0:
        #print('contraindication')
        return RELATION_CLASSES['contraindication']
    elif len(c)!=0 and len(a)==len(b)==len(d)==0:
        #print('relief')
        return  RELATION_CLASSES['relief']
    elif len(d)!=0 and len(a)==len(b)==len(c)==0:
        #print('treatment')
        return  RELATION_CLASSES['treatment']
    elif len(a)==len(b)==len(c)==len(d)==0:
        #print('IDK')
        return ABSTAIN
    return ABSTAIN

@labeling_function()
def phrase_closeness(x):
    text = x.Context
    pos1 = text.find(x.disease)
    dis_len = len(x.disease)
    classes, a, b, c, d = find_phrases(text)
    
    minDist = len(text.split())
    category = ABSTAIN
    for class_,found_phrases in {'effect':a, 'contraindication':b, 'relief':c, 'treatment':d}.items():
        for phrase_index in found_phrases:
            phrase =phrases.loc[phrase_index,class_]
            pos2 = text.find(phrase)
            phr_len = len(phrase)
            if pos1 > pos2:
                span = text[pos2+phr_len:pos1]
            else:
                span = text[pos1+dis_len:pos2]
            words = span.split()
            distance = len(words)
            if minDist > distance:
                minDist = distance
                category = class_
    
    if category == ABSTAIN:
        return ABSTAIN
    return RELATION_CLASSES[category]

@labeling_function()
def crowd(x):
    #print (x.relation_type)
    if x.relation in RELATION_CLASSES:
        return RELATION_CLASSES[x.relation]
    return ABSTAIN


In [49]:
ABSTAIN = -1

RELATION_CLASSES = {
'effect': 0,
'contraindication': 1,
'relief': 2,
'treatment':3 
} 
RELATION_CLASSES

{'effect': 0, 'contraindication': 1, 'relief': 2, 'treatment': 3}

In [50]:
from snorkel.labeling import PandasLFApplier

lfs = [crowd, phrases_claas, phrase_closeness]
applier = PandasLFApplier(lfs)

In [53]:
L_train = applier.apply(train_df)

100%|██████████| 40516/40516 [05:03<00:00, 133.60it/s]


In [51]:
L_dev = applier.apply(dev_df)

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 281/281 [00:02<00:00, 130.50it/s]


In [52]:
Y_dev.unique()

array([3, 0, 2, 1])

In [54]:
from snorkel.labeling import LabelModel
# Train LabelModel.
label_model = LabelModel(cardinality=4, verbose=True)
label_model.fit(L_train, n_epochs=100, seed=123, log_freq=20, l2=0.1, lr=0.01)

In [55]:
from snorkel.analysis import metric_score

preds_dev = label_model.predict(L_dev)

In [56]:
acc = metric_score(Y_dev, preds_dev, probs=None, metric="accuracy")
acc

0.800711743772242

In [57]:
LFAnalysis(L_dev, lfs).lf_summary(Y_dev.values).head()

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
crowd,0,"[0, 1, 2, 3]",0.501779,0.473310,0.209964,84,57,0.595745
phrases_claas,1,"[0, 1, 2, 3]",0.576512,0.576512,0.156584,142,20,0.876543
phrase_closeness,2,"[0, 1, 2, 3]",0.946619,0.743772,0.209964,219,47,0.823308


# New

In [294]:
from snorkel.labeling import filter_unlabeled_dataframe

probs_train = label_model.predict_proba(L_train)
pred_train =label_model.predict(L_train)
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=train_df, y=pred_train, L=L_train
)

In [295]:
df_train_filtered['noisy_labels'] =probs_train_filtered 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [296]:
df_train_filtered.relation = df_train_filtered.relation.replace(RELATION_CLASSES)

/home/jovyan/.local/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [297]:
df_train_filtered.rename(columns={'relation':'crowd_label'}, inplace=True)
df_train_filtered.to_csv('train_data.csv',index=False)

/home/jovyan/.local/lib/python3.7/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [298]:
test_df['label']= Y_dev
test_df.rename(columns={'relation':'crowd_label'}, inplace=True)
test_df.to_csv('test_data.csv',index=False)

In [301]:
len(df_train_filtered)

38772